# Метрические алгоритмы. Практика

В этом домашнем задании вы будете решать задачу классификации бутылок вина по различным характеристикам.

## Импорт библиотек, установка константных значений

In [1]:
import pandas as pd
import numpy as np

In [3]:
RANDOM_STATE = 42
TRAIN_SIZE = 0.75

In [4]:
rng = np.random.default_rng(RANDOM_STATE)

## Загрузка данных

In [5]:
from sklearn.datasets import load_wine

data = load_wine(as_frame=True)

X = data.data
y = data.target

## Задание 1

Посмотрите на количество классов и количество объектов каждого класса в датасете.

**Вопрос**:
Сколько классов в задаче?

In [8]:
# ваш код здесь
y.value_counts()

1    71
0    59
2    48
Name: target, dtype: int64

## Задание 2

Мы имеем дело с многоклассовой классификацией. Кроме того, классы не очень хорошо сбалансированы, поэтому для оценки качества модели метрика *accuracy* не подойдет.

Разбейте данные на тренировочную и тестовую части:
тестовая часть - 25% от всех данных, зафиксируйте `random_state = RANDOM_STATE`.

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=RANDOM_STATE, train_size=TRAIN_SIZE)

**Вопрос:**

Все ли признаки в данных одного масштаба?
Проверьте это, выведя основные числовые характеристики матрицы `X_train` методом `describe` из библиотеки `pandas`.

По полученной таблице числовых характеристик определите, какой признак измеряется в сотнях?
(если вариантов несколько, выберите признак с наибольшим средним значением).

In [10]:
# ваш код здесь
X_train.describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
count,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000
mean,12.972857,2.386842,2.362180,19.433835,100.759398,2.277068,2.021203,0.363534,1.608647,5.017594,0.959444,2.610000,742.992481
std,0.829993,1.098905,0.280606,3.467312,14.999571,0.645696,1.005537,0.126923,0.576964,2.202516,0.234545,0.729961,306.867593
min,11.030000,0.890000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.420000,1.740000,0.480000,1.270000,278.000000
25%,12.290000,1.640000,2.210000,17.200000,88.000000,1.700000,1.200000,0.260000,1.250000,3.250000,0.780000,1.830000,500.000000
50%,12.990000,1.900000,2.360000,19.400000,98.000000,2.230000,2.140000,0.340000,1.560000,4.800000,0.970000,2.810000,675.000000
75%,13.690000,3.170000,2.540000,21.500000,108.000000,2.800000,2.880000,0.450000,1.960000,6.130000,1.120000,3.200000,970.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,10.800000,1.710000,4.000000,1547.000000


## Задание 3

KNN требует того, чтобы все признаки были одного масштаба, поэтому масштабируйте данные при помощи `StandardScaler`.

Напоминаем, что обучать метод нужно только по тренировочным данным, а применять и к трейну, и к тесту.

После применения `StandardScaler` преобразуйте `X_train` и `X_test` к типу `pd.DataFrame`, названия новых объектов оставьте `X_train` и `X_test`.

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

In [13]:
X_train.describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
count,1.330000e+02,1.330000e+02,1.330000e+02,1.330000e+02,1.330000e+02,1.330000e+02,1.330000e+02,1.330000e+02,1.330000e+02,1.330000e+02,1.330000e+02,1.330000e+02,1.330000e+02
mean,-1.616084e-15,1.857328e-17,1.794721e-15,5.175476e-17,-3.245107e-16,-1.961672e-17,-4.487847e-16,5.290254e-16,5.509377e-17,-2.320617e-16,3.038505e-16,1.803069e-16,1.919935e-17
std,1.003781e+00,1.003781e+00,1.003781e+00,1.003781e+00,1.003781e+00,1.003781e+00,1.003781e+00,1.003781e+00,1.003781e+00,1.003781e+00,1.003781e+00,1.003781e+00,1.003781e+00
min,-2.349661e+00,-1.367272e+00,-3.584985e+00,-2.557379e+00,-2.058438e+00,-2.016383e+00,-1.678267e+00,-1.846925e+00,-2.067963e+00,-1.493740e+00,-2.051875e+00,-1.842656e+00,-1.521016e+00
25%,-8.258367e-01,-6.821936e-01,-5.443776e-01,-6.466911e-01,-8.538670e-01,-8.970924e-01,-8.197687e-01,-8.188072e-01,-6.239601e-01,-8.055682e-01,-7.679649e-01,-1.072591e+00,-7.948417e-01
50%,2.073230e-02,-4.446998e-01,-7.799877e-03,-9.795052e-03,-1.846607e-01,-7.317002e-02,1.185895e-01,-1.861196e-01,-8.463358e-02,-9.916689e-02,4.517819e-02,2.750233e-01,-2.224071e-01
75%,8.673013e-01,7.153658e-01,6.360934e-01,5.981512e-01,4.845456e-01,8.129352e-01,8.572970e-01,6.838259e-01,6.112716e-01,5.069710e-01,6.871333e-01,8.113187e-01,7.425540e-01
max,2.245999e+00,3.117706e+00,3.104351e+00,3.058886e+00,4.098260e+00,2.491871e+00,3.053455e+00,2.344631e+00,3.429688e+00,2.635290e+00,3.212157e+00,1.911412e+00,2.629953e+00


Обучите KNN с параметрами по умолчанию на тренировочных данных и сделайте предсказание на тесте.

In [28]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Будем измерять качество модели по метрике weighted $f1$-score.

Чтобы выбрать тип усреднения (micro, macro, weighted) в функции `f1_score` необходимо задать этот тип в гиперпараметре `average`.

Вычислите $f1$-score на тестовых данных.

**Вопрос:**

Чему равен $f1$-score на тестовых данных?

In [29]:
from sklearn.metrics import f1_score

score = f1_score(y_test, y_pred, average='weighted')
score

0.9550512333965844

## Задание 4

Попробуем улучшить модель.

Подберите оптимальное количество соседей (`n_neigbors`) из диапазона *от 3 до 30 с шагом 2* и веса соседей (`weights`):
`uniform`, `distance` по кросс-валидации с тремя фолдами на тренировочных данных.

Используйте `GridSearchCV` и метрику `f1_weighted`.

In [21]:
np.arange(3, 31, 2)

array([ 3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29])

In [24]:
from sklearn.model_selection import GridSearchCV

params = {'n_neighbors': np.arange(3, 30, 2), 'weights': ['uniform', 'distance']}
gs = GridSearchCV(KNeighborsClassifier(), param_grid=params, scoring='f1_weighted', cv=3, verbose=2, n_jobs=-1)
gs.fit(X_train, y_train)

print(gs.best_params_)
print(gs.best_score_)


Fitting 3 folds for each of 28 candidates, totalling 84 fits
{'n_neighbors': 19, 'weights': 'distance'}
0.9623290498688744


Возьмите best_estimator_, полученный при обучении GridSearchCV и с помощью него
сделайте предсказание на тесте и вычислите метрику `f1_weighted`.

In [30]:
best_knn = gs.best_estimator_
best_knn.fit(X_train, y_train)
y_pred = best_knn.predict(X_test)
score = f1_score(y_test, y_pred, average='weighted')
score

0.9550512333965844

**Вопрос:**

Удалось ли при помощи подбора гиперпараметров улучшить качество модели на тестовых данных?

## Задание 5

Выведите на экран матрицу ошибок.

Используйте модель с подобранными при помощи `GridSearch` гиперпараметрами.


**Вопрос:**
По этой матрице определите, какие классы между собой путает модель?

In [34]:
from sklearn.metrics import confusion_matrix

pd.DataFrame(confusion_matrix(y_test, y_pred), columns=[0, 1, 2])

,0,1,2
0,15,0,0
1,1,16,1
2,0,0,12


## Бонус (эксперименты с LSH)

Скопируйте все функции из [ноутбука в уроке "Быстрый поиск соседей"](https://colab.research.google.com/drive/181MMOcTnzdMVzJr0pWzqtEG0-BV9BIHH).

In [37]:
rng = np.random.default_rng(RANDOM_STATE)

In [ ]:
m = 100000
n = 10000   

In [ ]:
X = rng.normal(size=(m, n))  # датасет
q = rng.normal(size=n)       # вектор запроса

In [35]:
def knn_search(query, data, k=5):

    dists = np.sqrt(np.sum((data - query) ** 2, axis=1))  # вычисляем расстояния от объекта query до всех точек датасета
    inds = np.argsort(dists)  # сортируем по возрастанию расстояний
    inds_k = inds[:k]         # берем top-k точек с наименьшими расстояниями
    
    return data[inds_k], dists[inds_k]

In [36]:
def generate_hyperplanes(data, bucket_size=16):
    m = data.shape[0]            # число объектов
    n = data.shape[1]            # число признаков
    b = m // bucket_size         # количество корзин
    h = int(np.log2(b))          # количество гиперплоскостей
    H = rng.normal(size=(h, n))  # гиперплоскости, заданные своими нормалями
    return H

In [43]:
X_test.values

array([[ 8.06832067e-01,  6.51425164e-01,  7.07637133e-01,
        -1.22568751e+00,  1.01991065e+00,  6.57478171e-01,
         1.00703506e+00, -1.53058075e+00,  8.93427142e-02,
         3.75559480e-02,  2.38118688e-03,  1.03133729e+00,
         3.33672190e-01],
       [ 1.49618109e+00,  1.51005641e+00,  2.78374927e-01,
        -1.54544155e-01,  6.85307503e-01,  8.90663763e-01,
         6.27698755e-01, -5.02463378e-01, -6.23960098e-01,
         1.01359940e-01, -3.82791880e-01,  9.90083798e-01,
         1.10237008e+00],
       [-5.18307538e-02,  3.86528289e-01,  1.20844304e+00,
         4.53402076e-01, -3.18501945e-01, -1.14582374e+00,
        -1.51854612e+00,  1.31651351e+00, -1.49384158e+00,
        -1.90315452e-01, -8.10761955e-01, -4.12534916e-01,
        -4.67736251e-01],
       [ 9.15676650e-01, -8.10074836e-01,  1.20844304e+00,
         8.87649384e-01,  1.61012042e-02,  1.12384935e+00,
         1.22665081e+00, -5.81549330e-01,  1.34197204e+00,
         3.11001628e-01,  9.86712358e

In [54]:
results = knn_search(X_test.iloc[0].values, X_train.values, k=2)

In [55]:
results

(array([[ 1.50827494,  1.46438454,  0.52877788, -1.80468392,  1.82295821,
          1.12384935,  1.01701759, -1.2933229 ,  0.82004316,  0.03755595,
         -0.29719787,  1.26510707,  0.05563253],
        [ 0.92777049, -0.65479046, -0.40129023, -0.87828967,  1.15375191,
          0.50202111,  0.87726211, -1.21423695,  0.0197522 ,  0.37936305,
         -0.16880684,  0.81131867,  1.03694899]]),
 array([1.7873321 , 1.98096241]))

In [46]:
X_test.iloc[0].values

array([ 0.80683207,  0.65142516,  0.70763713, -1.22568751,  1.01991065,
        0.65747817,  1.00703506, -1.53058075,  0.08934271,  0.03755595,
        0.00238119,  1.03133729,  0.33367219])

При помощи `knn_search` найдите ближайших соседей к вину `X_test.iloc[0]` в **тренировочных** данных.

Обратите внимание, что функция `knn_search` принимает на вход `np.array`, а не `pd.DataFrame`. Поэтому переведите аргументы в `np.array`, приписав к необходимому объекту $X$: `X.values`.

In [ ]:
%%time

# ваш код здесь

Выведите на экран признаки объекта `X_test.iloc[0]` и признаки ближайшего найденного соседа.

In [ ]:
# ваш код здесь

**Вопрос:**

Можно ли сказать, что в тренировочных данных есть вино, почти такое же как `X_test.iloc[0]`? (все признаки почти одинаковые)

Какое расстояние между объектом запроса и первым ближайшим соседом?

Теперь найдите ближайшего соседа при помощи `approx_knn_search`.

In [ ]:
%%time

# ваш код здесь

Ближайший сосед при помощи KNN+LSH может быть найден не точно или не с первого запуска.
Запустите последнюю ячейку несколько раз и убедитесь, что ближайший сосед находится верно за несколько запусков.

**Вопрос:**

Запустите `knn_search` и `appox_knn_search` несколько раз и сравните время запусков. Какой из подходов в этой задаче работает быстрее?